In [93]:
import ee, geemap

In [94]:
ee.Initialize()

In [236]:
def maskL8sr(col):
    shadowsMask = (1 << 3)
    cloudMask = (1 <<5)
    qa = col.select('pixel_qa')
    mask = qa.bitwiseAnd(shadowsMask).eq(0).And(qa.bitwiseAnd(cloudMask).eq(0))
    return col.updateMask(mask)

In [237]:
# geometry = ee.Geometry.Polygon([[
#    [33.58, 5.54], [ 33.90,-0.94], [ 39.22,-4.72],[41.60,-1.61], [ 41.90,4.00], [36.18,5.77],[33.54,5.54]
# ]])
kenya = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na','Kenya')).geometry()

In [238]:
# col = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2020-04-01','2020-05-01').map(maskL8sr).filterBounds(kenya).median()

In [239]:
map1 = geemap.Map()
# map1.addLayer(col)
# map1

In [240]:
# image = col.clip(kenya)
# map1.addLayer(image)
# map1

In [282]:
vizParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 4000,
    'gamma': [1, 0.9, 1.1]
};

vizParams2 = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
};

In [283]:
col = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .map(maskL8sr) \
        .filterDate('2018-03-01','2021-10-1') \
        .filterBounds(kenya)
# .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 34))


In [284]:
image = col.median().clip(kenya)

In [285]:
map2 = geemap.Map()
map2.addLayer(image,vizParams2);
map2

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [296]:
ndvi1 = image.normalizedDifference(['B5','B4']).rename('NDVI')

In [297]:
col_list = col.toList(col.size())

In [298]:
def colEle(ele):
    date = ee.Image(ele).get('system:time_start')
    ndvi = ee.Image(ele).normalizedDifference(['B5', 'B4']).rename('NDVI');
    
    mini = ee.Number(ndvi.reduceRegion(
    reducer= ee.Reducer.min(),
    geometry= kenya,
    scale= 200,
    bestEffort=True).values().get(0))

    maxi = ee.Number(ndvi.reduceRegion(
        reducer= ee.Reducer.max(),
        geometry= kenya,
        scale= 200,
        bestEffort=True).values().get(0))
                     
    fv =(ndvi.subtract(mini).divide(maxi.subtract(mini))).pow(ee.Number(2)).rename('FV')
    a= ee.Number(0.004)
    b= ee.Number(0.986)
    EM=fv.multiply(a).add(b).rename('EMM')
                     
    image = ee.Image(ele);
                     
    LST = image.expression(
    '(Tb/(1 + (0.00115* (Tb / 1.438))*log(Ep)))-273.15', {
      'Tb': image.select('B10').multiply(0.1),
      'Ep': fv.multiply(a).add(b)
    })
    
    return ee.Algorithms.If(min, LST.set('system:time_start', date).float().rename('LST'), 0)

In [300]:
LST_col = col_list.removeAll([0])

In [301]:
LST_col = ee.ImageCollection(LST_col)

In [302]:
ndviParams = {'min': 0.10554729676864096, 'max': 0.41295681063122924,
    'palette': [
    'blue', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
    '74A901', '66A000', '529400', '3E8601', '207401', '056201',
    '004C00', '023B01', '012E01', '011D01', '011301']
}

map2.addLayer(ndvi1, ndviParams, 'ndvi1');
map2


Map(bottom=8536.0, center=[-0.9667509997666298, 38.12255859375001], controls=(WidgetControl(options=['position…

In [320]:
# select thermal band 10(with brightness tempereature), no calculation 
thermal= image.select('B10').multiply(0.1);
b10Params = {
    'min': 291.918, 
    'max': 302.382,
    'palette': [
'040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
'0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
'3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
'ff0000', 'de0101', 'c21301', 'a71001', '911003'
]
#     'blue', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
#     '74A901', '66A000', '529400', '3E8601', '207401', '056201',
#     '004C00', '023B01', '012E01', '011D01', '011301']
}
    
map2.addLayer(thermal, b10Params, 'thermal')
map2

Map(bottom=8536.0, center=[-0.9667509997666298, 38.12255859375001], controls=(WidgetControl(options=['position…

In [309]:
min1 = ee.Number(ndvi1.reduceRegion(
  reducer= ee.Reducer.min(),
  geometry= geometry,
  scale=30,
  maxPixels= 1e9
).values().get(0));



max1 = ee.Number(ndvi1.reduceRegion(
  reducer=ee.Reducer.max(),
  geometry= geometry,
  scale= 30,
  maxPixels= 1e9
).values().get(0));

In [310]:
fv = (ndvi1.subtract(min1).divide(max1.subtract(min1))).pow(ee.Number(2)).rename('FV')

In [311]:
a= ee.Number(0.004);
b= ee.Number(0.986);
EM = fv.multiply(a).add(b).rename('EMM');

imageVisParam3 = {min: 0.9865619146722164, max:0.989699971371314};

In [317]:
# var LST = col.map(function (image){

#   var date = image.get('system:time_start');
  
#   var LST = image.expression(
#     '(Tb/(1 + (0.00115* (Tb / 1.438))*log(Ep)))-273.15', {
#     'Tb': thermal.select('B10'),
#     'Ep':EM.select('EMM')
#   }).float().rename('LST');
  
#   return LST.set('system:time_start', date);
def colEle1(ele):
    date = ee.Image(ele).get('system:time_start')
    ndvi = ee.Image(ele).normalizedDifference(['B5', 'B4']).rename('NDVI');
    LST = image.expression(
    '(Tb/(1 + (0.00115* (Tb / 1.438))*log(Ep)))-273.15', {
    'Tb': thermal.select('B10'),
    'Ep':EM.select('EMM')}).float().rename('LST')
    return LST.set('system:time_start', date)
    
LST = col.map(colEle1)

In [319]:
map67 = geemap.Map()
map67.addLayer(LST.reduce(ee.Reducer.mean()))

KeyboardInterrupt: 

In [ ]:
map67

In [206]:
# LST in Celsius Degree bring -273.15
# NB: In Kelvin don't bring -273.15
LST = thermal.expression(
'(Tb/(1 + (0.00115* (Tb / 1.438))*log(Ep)))-273.15', {
 'Tb': thermal.select('B10'),
'Ep': EM.select('EMM')
}).rename('LST');

In [208]:
map2.addLayer(LST, {'min': 12.1, 'max':41.3, 'palette': [
'040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
'0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
'3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
'ff0000', 'de0101', 'c21301', 'a71001', '911003'
 ]},'LST');

In [229]:
task = ee.batch.Export.image.toDrive(LST, **{
    'description': 'LST',
    'folder': 'KituiLST',
    'region':geometry,
    'fileFormat': 'GeoTIFF',
    'scale':500,
    'formatOptions':{'cloudOptimized':True}
})

In [230]:
task.start()

In [33]:
# ndvi = image.normalizedDifference(['B5','B4']).rename('NDVI')

In [34]:
# MIN = ee.Number(ndvi.reduceRegion({
#     'reducer': ee.Reducer.min(),
#     'geometry': kenya,
#     'scale':30,
#     'maxPixels': 1e9
# }).values().get(0))

In [35]:
# PV = (ndvi.subtract(min).divide(max.subtract(min))).pow(ee.Number(2)).rename('FV')

In [36]:
# a = ee.Number(0.001490162)
# b = ee.Number(0.984809838)
# Emisivity = PV.multiply(a).add(b).rename('EM')
# image = ee.Image(ele)

In [37]:
# LST = image.expression(
#     '(Tb/(1+ (0.000115 * (Tb / 1.438)) * log(Ep)))-273.15',{
#         'Tb':image.select('B10').multiply(0.1),
#         'Ep':PV.multiply(a).add(b)
#     }
# )

In [ ]:
# result = ee.Algorithms.If(min, LST.set('system:time_start',date).float()rename('LST'),0)

In [ ]:
# LST_col = ee.ImageCollection(LST_col)

In [ ]:
# export_collection = LST_col.select(['LST']).toBands()

In [ ]:
# ee.Batch.Export.toDrive({
#     'image': export_collection,
#     'description': 'LST',
#     'folder': ' KituiLST',
#     'maxPixels':1e13,
#     'region':geometry,
#     'fileFormat': 'GeoTIFF',
#     'scale':30
# })